# Import modules

In [2]:
import numpy as np
import pingouin as pg
import pandas as pd
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy


/usr/local/lib/python3.8/dist-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Functions (helpers)

In [3]:
def read_csv(name, info=True, sep=','):
    df = pd.read_csv(f'../data/demographics/{name}.csv', sep=sep)
    # pd.set_option('display.max_columns', None)
    # pd.set_option('max_columns', None)
    if info:
        print_section('DATASET INFO')
        print('N rows=', df.size)
        print('N cols=', len(df.columns))
        print(df.head())
    return df

def print_section(title):
    print('\n')
    print( '*' + '-' * 135 + '*')
    print('* ' + title)
    print( '*' + '-' * 135 + '*')

def print_subsection(title):
    print('\n')
    print('***** ' + title + ' *****')
    print( '*' + '-' * 135 + '*')

In [13]:
filenames = [
   'interleaved_incomplete', 'block_complete',
   'block_complete_simple',
   'block_complete_mixed',  'block_complete_mixed_2s','block_complete_mixed_2s_amb_heuristic',
   'block_complete_mixed_2s_amb_final', 'incentivize', 'evoutcome'

]
dem = {}
for f in filenames:
    if f == 'block_complete_mixed_2s_amb_final':
        dem[f] = pd.read_csv('../data/demographics/' + f + '.csv', sep=';')
    else: dem[f] = read_csv(f)

# d = dem["block_incomplete"][dem['block_incomplete']['status'] == 'APPROVED']
# d
df = {}
ids = pd.read_csv('../sub_ids.csv')
ids['prolific_id'] = 'nothing'
for f in filenames:
    df = pd.read_csv('../data/csv/' + f + '.csv', header=None)
    # df.loc[-1] = [str(i) for i in range(len(df.columns))]  # adding a row
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True) 
    for i in ids[ids.exp_name==f]['id']:
        try:
            prolific_id = df[df[1]==i][2].unique()[0]
        # break
            ids.loc[ids.id==i, 'prolific_id'] = prolific_id
        except:
            try:
                prolific_id = df[df[1]==str(i)][2].unique()[0]
                ids.loc[ids.id==i, 'prolific_id'] = prolific_id
            except:
                print(i, ' is missing...')
    
# for i in ids['id']:
    # df
ids.to_csv('../sub_ids_with_prolific_id.csv', index=False)




*---------------------------------------------------------------------------------------------------------------------------------------*
* DATASET INFO
*---------------------------------------------------------------------------------------------------------------------------------------*
N rows= 2760
N cols= 20
                 session_id            participant_id    status  \
0  5d30c987983db9001aee0190  5d30b98eb8f4a500193ce6c6  APPROVED   
1  5d30c9b8aa269b00194c5ae0  596a460792973f0001edc027  RETURNED   
2  5d30c9c798a422001930c607  5d30c81f5f791a001fb95cae  APPROVED   
3  5d30c9ddae02c60019777caa  5c15987500b8ee0001d0e380  APPROVED   
4  5d30c9ed459d15000139e59f  5d30c6a98c13a90015651c44  APPROVED   

             started_datetime         completed_date_time    time_taken   age  \
0  2019-07-18 19:33:30.822000  2019-07-18 20:04:30.531000  1.859709e+03  21.0   
1  2019-07-18 19:34:25.817000                         NaN  6.150167e+07  56.0   
2  2019-07-18 19:34:33.730000  2019-0

/home/getz/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11,12,13,14,15,18,19,20,21,22,23,24,25,26,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [29]:
df = pd.read_csv('../sub_ids_with_prolific_id.csv')

ids = df['prolific_id'].unique()
exps = df['exp_name'].unique()
df['age'] = np.nan
df['gender'] = np.nan
for f in exps:
    sep = ','
    if f == 'block_incomplete':
        continue
    if f == 'block_complete_mixed_2s_amb_final':
        sep = ';'
    print(f)
    dem = pd.read_csv('../data/demographics/' + f + '.csv', sep=sep)
    for i in ids:
        try:
            df.loc[df.prolific_id==i, 'gender'] = dem.loc[dem['participant_id'] == i, 'Sex'].values[0]
            df.loc[df.prolific_id==i, 'age'] = dem.loc[dem['participant_id'] == i, 'age'].values[0]
        except:
            # print(i, ' is missing...')
            pass
df

interleaved_incomplete
block_complete
block_complete_simple
block_complete_mixed
block_complete_mixed_2s
block_complete_mixed_2s_amb_heuristic
block_complete_mixed_2s_amb_final
incentivize
evoutcome


,id,exp_num,exp_name,prolific_id,age,gender
0,76422442,1,interleaved_incomplete,5d30c0f46cc6bd0020882925,26.0,Female
1,192489031,1,interleaved_incomplete,5d3ebe453a0c0100162313b9,31.0,Female
2,324224518,1,interleaved_incomplete,5d3ebb0d64c213001ad49fb0,24.0,Female
3,543099212,1,interleaved_incomplete,5cd875edf7470b0016e9ca83,29.0,Female
4,635786936,1,interleaved_incomplete,5d0bccb311a6b5001588347b,63.0,Male
...,...,...,...,...,...,...
748,9049475133,10,evoutcome,5f1e9db721e4f101783fe21c,21,Male
749,9195332330,10,evoutcome,61119ee3576ae2aa2cc8cc32,19,Female
750,9391982197,10,evoutcome,5f68f02c41716709ee9be42d,25,Male
751,9484643174,10,evoutcome,615767861330080eb8cad21c,36,Female
